# Литература

* Увеличение чувствительности A/Б-тестов с помощью Cuped. Доклад в Яндексе
[статья](https://habr.com/ru/companies/yandex/articles/497804/)
[видео](https://www.youtube.com/watch?v=pZpUM08mv-E)

* [Как улучшить ваши A/B-тесты: лайфхаки аналитиков Авито. Часть 2](https://habr.com/ru/companies/avito/articles/571096/)

In [1]:
# Сгенерить данные для двух групп для двух временных промежутков

In [9]:
from collections import namedtuple
import scipy.stats as sps
import statsmodels.stats.api as sms
from tqdm.notebook import tqdm as tqdm_notebook # tqdm – библиотека для визуализации прогресса в цикле
from collections import defaultdict
from statsmodels.stats.proportion import proportion_confint
import numpy as np
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.5, palette='Set2')
ExperimentComparisonResults = namedtuple('ExperimentComparisonResults', 
                                        ['pvalue', 'effect', 'ci_length', 'left_bound', 'right_bound'])

In [11]:
def absolute_ttest(control, test):
    mean_control = np.mean(control)
    mean_test = np.mean(test)
    var_mean_control  = np.var(control) / len(control)
    var_mean_test  = np.var(test) / len(test)
    
    difference_mean = mean_test - mean_control
    difference_mean_var = var_mean_control + var_mean_test
    difference_distribution = sps.norm(loc=difference_mean, scale=np.sqrt(difference_mean_var))

    left_bound, right_bound = difference_distribution.ppf([0.025, 0.975])
    ci_length = (right_bound - left_bound)
    pvalue = 2 * min(difference_distribution.cdf(0), difference_distribution.sf(0))
    effect = difference_mean
    return ExperimentComparisonResults(pvalue, effect, ci_length, left_bound, right_bound)



# 2. Создание тестируемого критерия
def cuped_ttest(control, test, control_before, test_before):
    theta = (np.cov(control, control_before)[0, 1] + np.cov(test, test_before)[0, 1]) /\
                (np.var(control_before) + np.var(test_before))

    control_cup = control - theta * control_before
    test_cup = test - theta * test_before
    return absolute_ttest(control_cup, test_cup)

In [12]:
# 2. Создание тестируемого критерия.
def cuped_ttest(control, test, control_before, test_before):
    theta = (np.cov(control, control_before)[0, 1] + np.cov(test, test_before)[0, 1]) /\
                (np.var(control_before) + np.var(test_before))

    control_cup = control - theta * control_before
    test_cup = test - theta * test_before
    return absolute_ttest(control_cup, test_cup)
  
# 3. Заводим счётчик.
bad_cnt = 0

# 4. Цикл проверки.
N = 30000
for i in range(N):
    # 4.a. Тестирую A/B-тест.
    control_before = sps.expon(scale=1000).rvs(1000)
    control = control_before + sps.norm(loc=0, scale=100).rvs(1000)

    test_before = sps.expon(scale=1000).rvs(1000)
    test = test_before + sps.norm(loc=0, scale=100).rvs(1000)
    test *= 1.1

    # 4.b. Запускаю критерий.
    _, _, _, left_bound, right_bound = cuped_ttest(control, test, control_before, test_before)
    
    # 4.c. Проверяю, лежит ли истинная разница средних в доверительном интервале.
    if left_bound > 100 or right_bound < 100:
        bad_cnt += 1
        
# 5. Строю доверительный интервал для конверсии ошибок у критерия.
left_real_level, right_real_level = proportion_confint(count = bad_cnt, nobs = N, alpha=0.05, method='wilson')
# Результат.
print(f"Реальный уровень значимости: {round(bad_cnt / N, 4)};"
      f" доверительный интервал: [{round(left_real_level, 4)}, {round(right_real_level, 4)}]")

Реальный уровень значимости: 0.0526; доверительный интервал: [0.0501, 0.0552]
